In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=50,50
plt.rcParams.update({'figure.max_open_warning': 0})
%matplotlib inline

In [ ]:
from sklearn_pandas import CategoricalImputer
from sklearn import preprocessing 
from sklearn.preprocessing import Imputer

In [ ]:
import seaborn as sns
color = sns.color_palette()

In [ ]:
#Datatypes definition
dtypes = {
        'MachineIdentifier':                                    'str',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

**Loading Data**

In [ ]:
#Data reading using dask
import dask
import dask.dataframe as dd
train_data = dd.read_csv('../input/train.csv', dtype=dtypes)
test_data = dd.read_csv('../input/test.csv', dtype=dtypes)

In [ ]:
train_data.head(5)

In [ ]:
test_data.head(5)

In [ ]:
train_data = train_data.compute()  
test_data = test_data.compute()

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
submit_df = test_data[['MachineIdentifier']]

**Missing Data**

In [ ]:
#missing data
def display_missing_data(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data.head(40))

In [ ]:
display_missing_data(train_data)

In [ ]:
display_missing_data(test_data)

In [ ]:
train_data.set_index('MachineIdentifier', inplace=True)

In [ ]:
cols_drop =['PuaMode','Census_ProcessorClass','DefaultBrowsersIdentifier','Census_IsFlightingInternal',
            'Census_InternalBatteryType','Census_ThresholdOptIn','Census_IsWIMBootEnabled']

In [ ]:
def drop_missing_value_columns():
    train_data.drop(cols_drop, axis=1,inplace=True)
    test_data.drop(cols_drop, axis=1,inplace=True)

In [ ]:
drop_missing_value_columns()

*Check the each data type of column  and convert the date type to a category or int *

In [ ]:

train_data['Census_ProcessorCoreCount'] = train_data['Census_ProcessorCoreCount'].fillna(0).apply(np.int8)

In [ ]:
train_data['Census_TotalPhysicalRAM'] = train_data['Census_TotalPhysicalRAM'].fillna(0).apply(np.int32)

**Exploratory Data Analysis**

**Continuous  Variable Analysis**

1. Histogram of variable
2. Find out whether is it right data type . It can be converted to int or bool based on the values
3. Check the boxplot and find out if there are any outliers
4. There are numeric variables which can be converted to category and then to label encoding
5. populate missing values
6. Draw historgram 
7. Calculate the correlation and decide whether to consider in the model or not
8. Convert data type


In [ ]:
train_data_continuous = train_data.select_dtypes([np.int8,np.int16,np.int32,np.float16,np.float32])

*The below mehtod will get the unique category column count*

In [ ]:
def unique_category_count(colname):
    print(train_data[colname].nunique()) 

*The below method will populate the missing values in category column with most frequent value*

In [ ]:
def populate_missing_category_values(colname):
    cat_imputer = CategoricalImputer()
    train_data[colname]= cat_imputer.fit_transform(train_data[colname]).ravel()
    test_data[colname]= cat_imputer.fit_transform(test_data[colname]).ravel()

*The below method will encode the category values with number *

In [ ]:
def label_encode_category_values(colname):
    lbl_encoder = preprocessing.LabelEncoder()
    train_data[colname] = lbl_encoder.fit_transform(train_data[colname].astype(str))
    test_data[colname] = lbl_encoder.fit_transform(test_data[colname].astype(str))

In [ ]:
def calculate_corr_value(colname):
    corr_val = train_data['HasDetections'].corr(train_data[colname])
    print('The correlation between HasDetections variable and '+ colname +':'+ str(corr_val))

*The below method will populate the missing values in column with median value*

In [ ]:
def populate_missing_numerical_values(colname):
    imputer = Imputer(strategy = 'median')
    train_data[colname]= imputer.fit_transform(train_data[[colname]]).ravel()
    test_data[colname]= imputer.fit_transform(test_data[[colname]]).ravel()
    

In [ ]:
def drop_column(colname):
    train_data.drop(colname, axis=1,inplace=True)
    test_data.drop(colname, axis=1,inplace=True)

In [ ]:
def draw_histogram(colname):
    train_data[colname].hist()

In [ ]:
def draw_category_histogram(colname):
    train_data[colname].value_counts(sort=False).sort_values(ascending=False).nlargest(50).plot(kind='bar', figsize=[20,10],title=col)

In [ ]:
def display_valuecount_bytarget_variable(colname):
    train_data.groupby(['HasDetections',colname])[colname].count()

In [ ]:
def draw_box_plot(colname):
    train_data.boxplot(column=colname)

In [ ]:
def describe_numeric_data(colname):
    train_data[colname].describe()

In [ ]:
def outliers_iqr(colname):
        print(colname)
        Q1 = train_data[colname].quantile(0.25)
        Q3 = train_data[colname].quantile(0.75)
        print(Q1)
        print(Q3)
        IQR = Q3 - Q1
        print(IQR)
        df_clean = train_data[(train_data[colname] < (Q1 - 1.5 * IQR)) | (train_data[colname] > (Q3 + 1.5 * IQR))]
        return df_clean.shape[0]

In [ ]:
def draw_values_distribution_plot(colname):
    plt.figure(figsize=(8,4))
    sns.violinplot(x=train_data[colname], y=train_data['HasDetections'], data=train_data)
    plt.ylabel(colname, fontsize=12)
    plt.xlabel('HasDetections', fontsize=12)
    plt.title(colname+" distribution")
    plt.show()

*The correlation is negligible   and also the response variable percentaget for 1 value is  very low . So, we will not conside this value in our model*

In [ ]:
train_data_continuous = train_data.select_dtypes([np.int8,np.int16,np.int32,np.float16,np.float32])
corr_low_value_variables =[]
corr_high_value_variables =[]
corr_low_value_df=[]
corr_high_value_df=[]
for i, col in enumerate(train_data_continuous.columns):
    #print(train_data.groupby(['HasDetections',col])[col].count())
    corr_val = train_data['HasDetections'].corr(train_data[col])
    if(corr_val > 0.05  or corr_val < -0.05):
        print(col)
        corr_high_value_variables.append(col)
        corr_high_value_df.append((col ,corr_val))
    else:
        print('correlating with target variable is very less for variable name: '+ col)
        corr_low_value_variables.append(col)
        corr_low_value_df.append((col,corr_val))
    

In [ ]:
corr_low_Values_df= pd.DataFrame(corr_low_value_df, columns=('Variable Name','Corr value'))
corr_low_Values_df

In [ ]:
corr_high_value_df =pd.DataFrame(corr_high_value_df, columns=('Variable Name','Corr value'))
corr_high_value_df

*The variables with low value correlation are *

In [ ]:
print(corr_low_value_variables)

*The variables with high value correlation are *

In [ ]:
print(corr_high_value_variables)

*We will draw box plots to identify if there are any outliers*

In [ ]:
for i, col in enumerate(corr_high_value_variables):
    plt.figure(i)
    train_data.boxplot(column=col)


In [ ]:
draw_box_plot('AVProductStatesIdentifier')

In [ ]:
draw_box_plot('AVProductsInstalled')

In [ ]:
draw_box_plot('IsProtected')

In [ ]:
draw_box_plot('Census_IsVirtualDevice')

In [ ]:
draw_box_plot('Census_IsAlwaysOnAlwaysConnectedCapable')

In [ ]:
draw_box_plot('Wdft_IsGamer')

In [ ]:
describe_numeric_data('AVProductStatesIdentifier')

In [ ]:
display_valuecount_bytarget_variable('AVProductStatesIdentifier')

In [ ]:
draw_histogram('AVProductStatesIdentifier')

*We have outliers for the variables AVProductStatesIdentifier, Census_ProcessorCoreCount, Census_TotalPhysicalRAM*

*We will find the number of outliers for the above variables*

In [ ]:
outliers_iqr('AVProductStatesIdentifier')

**Category variable  Analysis**

1. The number of unique values in the column (no of categories)
2. Adding comment on missing values 
3. Draw histogram with value counts and also diplay the value counts of each category and target variable
4. populate missing values
5. Draw historgram with value counts and also diplay the value counts of each category and target variable
6. Calculate the correlation and decide whether to consider in the model or not
7. if yes , find out the distribution of each category and add inference comment.
8. if the values are very less , decide on binary , multiple category or one hot encoding

In [ ]:
train_data_cat = train_data.select_dtypes(['category'])

In [ ]:
train_data_cat.columns

In [ ]:
draw_category_histogram('ProductName')

In [ ]:
draw_category_histogram('EngineVersion')

In [ ]:
draw_category_histogram('AppVersion')

In [ ]:
draw_category_histogram('AvSigVersion')

In [ ]:
draw_category_histogram('Platform')

In [ ]:
draw_category_histogram('Processor')

*Find out the unique category value count for each category column *

In [ ]:
train_data['Census_ChassisTypeName'].unique() 

In [ ]:
for i, col in enumerate(train_data_cat.columns):
    print('The number of category values for the column : '+ col+ ' - '+str(len(list(train_data[col].unique()))))
    

In [ ]:
train_data['ProductName'].value_counts(sort=False).sort_values(ascending=False)

In [ ]:
train_data['Platform'].value_counts(sort=False).sort_values(ascending=False)

In [ ]:
train_data['Processor'].value_counts(sort=False).sort_values(ascending=False) 

*There are high cardinal category columns and we need to convert to numeric values by ordinal encoding.*

In [ ]:

corr_low_value_cat_variables =[]
corr_high_value_cat_variables =[]
corr_low_value_cat_df=[]
corr_high_value_cat_df=[]
from sklearn.preprocessing import LabelEncoder
for i, col in enumerate(train_data_cat.columns):
    lbl_encoder = LabelEncoder()
   
    #if(col in ('OsBuildLab ','SmartScreen','Census_PrimaryDiskTypeName')):
    train_data[col] = lbl_encoder.fit_transform(train_data[col].astype(str))
    test_data[col] = lbl_encoder.fit_transform(test_data[col].astype(str))
    #else:
      #  train_data_encoded[col] = lbl_encoder.fit_transform(train_data_encoded[col])
    corr_val = train_data['HasDetections'].corr(train_data[col])
    print(corr_val)
    if(corr_val > 0.05  or corr_val < -0.05):
        print(col)
        corr_high_value_cat_variables.append(col)
        corr_high_value_cat_df.append((col ,corr_val))
    else:
        print('correlating with target variable is very less for variable name: '+ col)
        corr_low_value_cat_variables.append(col)
        corr_low_value_cat_df.append((col,corr_val))

In [ ]:
corr_low_value_cat_df =pd.DataFrame(corr_low_value_cat_df, columns=('Variable Name','Corr value'))
corr_low_value_cat_df

In [ ]:
corr_high_value_cat_df = pd.DataFrame(corr_high_value_cat_df, columns=('Variable Name','Corr value'))
corr_high_value_cat_df

In [ ]:
train_data.head()

**Model Training**

In [ ]:
test_data.set_index('MachineIdentifier', inplace=True)

In [ ]:
train_data_labels = train_data['HasDetections']

In [ ]:
selected_columns =['EngineVersion','AvSigVersion','Processor','SmartScreen',
                                       'Census_OSArchitecture','AVProductStatesIdentifier','AVProductsInstalled','IsProtected',
                                       'Census_IsVirtualDevice','Census_IsAlwaysOnAlwaysConnectedCapable','Wdft_IsGamer']

In [ ]:
train_data = train_data[selected_columns]

In [ ]:
test_data = test_data[selected_columns]

In [ ]:
test_data.head()

In [ ]:
train_data.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Imputer
imputer = Imputer(strategy = 'median')
imputer.fit(train_data)
imputer.fit(test_data)
train_data = imputer.transform(train_data)
test_data =imputer.transform(test_data)


In [ ]:
type(train_data)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train_data, train_data_labels)

**Test Data Transformation**

*Test data will be transformed as per input training data and predict the output*

In [ ]:
log_reg_pred = log_reg.predict_proba(test_data)[:, 1]

In [ ]:
log_reg_pred.shape

In [ ]:
submit_df['HasDetections'] = log_reg_pred

In [ ]:
submit_df.head()

In [ ]:
submit_df.to_csv('submission.csv', index=False)